In [21]:
from html.parser import HTMLParser
import nltk
import numpy as np

In [22]:
class HtmlHandler(HTMLParser):
    def reset(self):
        self.inTitle = False
        self.inPara = False
        self.result = ""
        self.ret = ""
        HTMLParser.reset(self)
    
    def handle_starttag(self, tag, attrs):
        if tag == "title":
            self.inTitle = True
        elif tag == "p":
            self.inPara = True
        # print("Encountered a start tag:", tag)

    def handle_endtag(self, tag):
        if tag == "title":
            self.inTitle = False
        elif tag == "p":
            self.inPara = False
        # print("Encountered an end tag :", tag)

    def handle_data(self, data):
        pflag = False
        if self.inTitle or self.inPara:
            for c in data:
                if c != ' ' and c != '' and c != '\n' and c != '\t':
                    pflag = True
                    break
            if pflag:
                self.result += data
                self.result += " "
        # print("Encountered some data  :", data)
    
    def ret_result(self):
        for c in self.result:
            if c != '\n' and c != '\t':
                self.ret += c
        return self.ret

In [23]:
def html2vec(file_path, word2idx):
    '''
    parameters:
    file_path: input html file path
    word2idx: fetched from trained doc2vec model, see doc2vec_clean.ipynb
    
    return:
    ret_vector: unpadded vector, requires padding
    '''
    ret_vector = []
    english_punctuations = ['|', '-', '·', '``', ',', '.', ':', ';', '?', '(', ')', '[', ']', '<', '>', '&', '!', '*', '@', '#', '$', '%', '...', "'", '"', "--", "`", "“", "”", "’", "‘", "_", "''", '+', '*', '/', '\\', '=', '~', '^', '{', '}']
    Handler = HtmlHandler()
    Handler.reset()
    f = open(file_path, 'r')
    html = ""
    for tmp in f.readlines():
        tmp.strip('\n')
        html += tmp
    f.close()
    Handler.feed(html)
    result = Handler.ret_result()
    result = nltk.word_tokenize(result)
    result = [word.lower() for word in result if word not in english_punctuations]
    for word in result:
        if word in word2idx.keys():
            ret_vector.append(word2idx[word])
    return ret_vector